In [1]:
# !python --version

Python 3.6.9 :: Anaconda, Inc.


In [2]:
# !pip install ffmpeg-python

In [3]:
# Make sure to install this ffmpeg and add to system 
# linux command sudo apt  install ffmpeg

In [4]:
import os
path = "/media/shivam/New Volume1/dataset"
path_changer = "/"

In [5]:
from imutils import face_utils
import dlib
import sys
import cv2
import numpy as np
import ffmpeg

In [6]:
save_path = os.getcwd()

In [7]:
def check_rotation(path_video_file):
    # this returns meta-data of the video file in form of a dictionary
    meta_dict = ffmpeg.probe(path_video_file)
    if 'rotate' not in str(meta_dict):
        return None
    # from the dictionary, meta_dict['streams'][0]['tags']['rotate'] is the key
    # we are looking for
    rotateCode = None
    try:
        if int(meta_dict['streams'][1]['tags']['rotate']) == 90:
            rotateCode = cv2.ROTATE_90_CLOCKWISE
        elif int(meta_dict['streams'][1]['tags']['rotate']) == 180:
            rotateCode = cv2.ROTATE_180
        elif int(meta_dict['streams'][1]['tags']['rotate']) == 270:
            rotateCode = cv2.ROTATE_90_COUNTERCLOCKWISE
        return rotateCode
    except:
        if int(meta_dict['streams'][0]['tags']['rotate']) == 90:
            rotateCode = cv2.ROTATE_90_CLOCKWISE
        elif int(meta_dict['streams'][0]['tags']['rotate']) == 180:
            rotateCode = cv2.ROTATE_180
        elif int(meta_dict['streams'][0]['tags']['rotate']) == 270:
            rotateCode = cv2.ROTATE_90_COUNTERCLOCKWISE
        return rotateCode
    else:
        return None

In [8]:
def correct_rotation(frame, rotateCode):
    return cv2.rotate(frame, rotateCode) 

In [9]:
# p = our pre-treined model directory, on my case, it's on the same script's diretory.
p = "shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(p)

In [10]:
temp = os.listdir(path)

In [11]:
li = []
for i in temp:
    if ".zip" not in i:
        li.append(i)

In [12]:
li

['Fold1_part1',
 'Fold1_part2',
 'Fold2_part1',
 'Fold2_part2',
 'Fold3_part1',
 'Fold3_part2',
 'Fold4_part1',
 'Fold4_part2']

In [13]:
def printToFile(shape,file):
    #Write code to sace 68*2 points on matrix
    print_shape = shape.reshape(1,-1)
    print_shape = print_shape[0]
    for i in print_shape:
        file.write(str(i)+",")
    file.write("\n")

In [14]:
def fun(video_path,file):
    # Create a VideoCapture object and read from input file
    # If the input is the camera, pass 0 instead of the video file name
    cap = cv2.VideoCapture(video_path)
    # check if video requires rotation
    rotateCode = check_rotation(video_path)
    #check if camera opened successfully
    if cap.isOpened() == False:
        print("Error opening video stream or file")
    # Read until video is completed
    while cap.isOpened():
        ret,image = cap.read()
        if ret == True:
            if rotateCode is not None:
                image = correct_rotation(image, rotateCode)
            #Display Frame
            #cv2.imshow('frame',frame)
            # Converting the image to gray scale
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            # Get faces into webcam's image
            rects = detector(gray, 0)

            # For each detected face, find the landmark.
            for (i, rect) in enumerate(rects):
                # Make the prediction and transfom it to numpy array
                shape = predictor(gray, rect)
                shape = face_utils.shape_to_np(shape)
                printToFile(shape,file)
                # Draw on our image, all the finded cordinate points (x,y) 
                for (x, y) in shape:
                    cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
            # Show the image
            cv2.imshow("Output", image)
            #Press Q on keyboard to exit
            if cv2.waitKey(30) & 0xFF == ord('q'):
                break
        else:
            break
    cap.release()
    cv2.destroyAllWindows()

In [15]:
file_changer = "@"
for i in range(len(li)):
    folders = os.listdir(path+path_changer+li[i])
    for j in range(len(folders)):
        videos = os.listdir(path+path_changer+li[i]+path_changer+folders[j])
        for k in range(len(videos)):
            video = path+path_changer+li[i]+path_changer+folders[j]+path_changer+videos[k]
            file_name = li[0]+file_changer+folders[j]+file_changer+videos[k]+".txt" 
            file = open(os.path.join(save_path, file_name),"w")
            fun(video,file)
            file.close()